## Test du modèle NERCamemBERT de Loïck Bourdois
Fonctionne sur trois types d'entités : personnes (PER), lieux (LOC), organisations (ORG)

In [1]:
import pandas as pd
import re
from transformers import pipeline

def clean_entity_text(text):
    """Nettoie le texte d'une entité en supprimant les caractères spéciaux"""
    # Supprimer les caractères spéciaux
    text = re.sub(r'^[^\w\s]+|[^\w\s]+$', '', text)
    # Supprimer les caractères de ponctuation isolés
    text = re.sub(r'\s+[^\w\s]+\s+', ' ', text)
    # Normaliser les espaces
    text = ' '.join(text.split())
    return text

def filter_entities(entities, min_confidence=0.65):
    """Filtre les entités par confiance et nettoie le texte"""
    filtered_entities = []
    
    for entity in entities:
        # Vérifier la confiance
        if entity['score'] >= min_confidence:
            # Nettoyer le texte de l'entité
            cleaned_text = clean_entity_text(entity['word'])
            
            # Ne garder que si le texte nettoyé n'est pas vide et fait plus d'1 caractère
            if cleaned_text and len(cleaned_text) > 1:
                entity_copy = entity.copy()
                entity_copy['word'] = cleaned_text
                filtered_entities.append(entity_copy)
    
    return filtered_entities

def process_csv_ner(csv_file, min_confidence=0.65):
    """Traite tous les textes du CSV avec filtrage par confiance"""
    
    # Charger le pipeline NER
    print("Chargement du modèle NER...")
    ner = pipeline(
        'token-classification', 
        model='CATIE-AQ/Moderncamembert_3entities', 
        tokenizer='CATIE-AQ/Moderncamembert_3entities', 
        aggregation_strategy="simple"
    )
    
    # Charger le CSV
    print("Chargement du CSV...")
    try:
        df = pd.read_csv(csv_file, sep=';', encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(csv_file, sep=';', encoding='latin1')
    
    # Filtrer les textes valides qui ont un ID
    df_valid = df[df['texte'].notna() & (df['texte'].str.len() > 10) & df['id'].notna()].copy()
    
    print(f"Traitement de {len(df_valid)} textes...")
    print(f"Confiance minimum: {min_confidence}")
    
    # Résultats
    all_results = []
    
    for i, (idx, row) in enumerate(df_valid.iterrows()):
        text = row['texte']
        text_id = row['id']  # Récupérer l'ID du CSV original
        
        try:
            # Appliquer le NER
            raw_results = ner(text)
            
            # Filtrer par confiance et nettoyer
            filtered_results = filter_entities(raw_results, min_confidence)
            
            # Stocker les résultats avec text_id
            all_results.append({
                'text_index': idx,  # Index pandas (pour compatibilité)
                'text_id': text_id,  # ID du CSV original
                'text_length': len(text),
                'entities': filtered_results,
                'entities_raw_count': len(raw_results),
                'entities_filtered_count': len(filtered_results)
            })
            
            # Afficher progression tous les 50 textes
            if (i + 1) % 50 == 0:
                progress = ((i + 1) / len(df_valid)) * 100
                print(f"Progression: {i + 1}/{len(df_valid)} ({progress:.1f}%)")
                
        except Exception as e:
            print(f"Erreur texte {text_id} (index {idx}): {e}")
            all_results.append({
                'text_index': idx,
                'text_id': text_id,
                'text_length': len(text),
                'entities': [],
                'entities_raw_count': 0,
                'entities_filtered_count': 0
            })
    
    # Statistiques de filtrage
    total_raw = sum(r['entities_raw_count'] for r in all_results)
    total_filtered = sum(r['entities_filtered_count'] for r in all_results)
    print(f"Filtrage terminé: {total_raw} -> {total_filtered} entités (conf >= {min_confidence})")
    
    return all_results

def save_results_simple(results, output_file='ner_results.csv'):
    """Sauvegarde les résultats en CSV avec text_id"""
    
    data = []
    
    for result in results:
        text_id = result.get('text_id', result['text_index'])  # Utiliser text_id si disponible
        text_len = result['text_length']
        
        if result['entities']:
            for entity in result['entities']:
                data.append({
                    'text_id': text_id,
                    'text_length': text_len,
                    'entity': entity['word'],
                    'label': entity['entity_group'],
                    'confidence': entity['score'],
                    'start': entity['start'],
                    'end': entity['end']
                })
        else:
            # Ligne vide si pas d'entités
            data.append({
                'text_id': text_id,
                'text_length': text_len,
                'entity': '',
                'label': '',
                'confidence': 0,
                'start': 0,
                'end': 0
            })
    
    # Créer DataFrame et sauvegarder
    df_results = pd.DataFrame(data)
    df_results.to_csv(output_file, index=False, encoding='utf-8')
    
    print(f"Résultats sauvegardés dans {output_file}")
    
    # Statistiques
    entities_only = df_results[df_results['entity'] != '']
    total_entities = len(entities_only)
    per_count = sum(entities_only['label'] == 'PER')
    org_count = sum(entities_only['label'] == 'ORG')
    loc_count = sum(entities_only['label'] == 'LOC')
    
    print(f"Statistiques:")
    print(f"- Total entités: {total_entities}")
    print(f"- Personnes (PER): {per_count}")
    print(f"- Organisations (ORG): {org_count}")  
    print(f"- Lieux (LOC): {loc_count}")
    
    return df_results

# Usage simple
if __name__ == "__main__":
    csv_file = 'Candidatures58-81-290825.csv'
    
    print("\nAnalyse complète avec filtrage:")
    print("results = process_csv_ner(csv_file, min_confidence=0.65)")
    print("df_results = save_results_simple(results)")

print("Code avec filtrage par confiance et nettoyage prêt")

/Users/charlielezin/miniconda3/envs/camembert-ner/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/charlielezin/miniconda3/envs/camembert-ner/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/charlielezin/miniconda3/envs/camembert-ner/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <5AA8DD3D-A2CC-31CA-8060-88B4E9C18B09> /Users/charlielezin/miniconda3/envs/camembert-ner/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <29934C5C-3B9F-3E3A-B719-1C007C303C01> /Users/charlielezin/miniconda3/envs/camembert-ner/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. O


Analyse complète avec filtrage:
results = process_csv_ner(csv_file, min_confidence=0.65)
df_results = save_results_simple(results)
Code avec filtrage par confiance et nettoyage prêt


/Users/charlielezin/miniconda3/envs/camembert-ner/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/Users/charlielezin/miniconda3/envs/camembert-ner/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to

In [2]:
results = process_csv_ner(csv_file, min_confidence=0.65)

Chargement du modèle NER...


Device set to use mps:0


Chargement du CSV...
Traitement de 303 textes...
Confiance minimum: 0.65
Progression: 50/303 (16.5%)
Progression: 100/303 (33.0%)
Progression: 150/303 (49.5%)
Progression: 200/303 (66.0%)
Progression: 250/303 (82.5%)
Progression: 300/303 (99.0%)
Filtrage terminé: 5646 -> 4051 entités (conf >= 0.65)


In [3]:
df_results = save_results_simple(results)

Résultats sauvegardés dans ner_results.csv
Statistiques:
- Total entités: 4051
- Personnes (PER): 1432
- Organisations (ORG): 1062
- Lieux (LOC): 1557


## Métriques

## Métriques avant vérité terrain

In [4]:
from collections import Counter
import pandas as pd

def analyze_predictions_before_ground_truth(csv_results_file):
    """Analyse les prédictions du modèle NER avant comparaison avec la vérité terrain"""
    
    print("Chargement du fichier de résultats...")
    
    # Charger les résultats NER
    try:
        df_results = pd.read_csv(csv_results_file, encoding='utf-8')
    except UnicodeDecodeError:
        df_results = pd.read_csv(csv_results_file, encoding='latin1')
    
    print("Analyse des prédictions du modèle...")
    
    # Statistiques générales
    total_texts = df_results['text_id'].nunique()
    total_predictions = len(df_results[df_results['entity'].notna() & (df_results['entity'] != '')])
    
    print(f"\nSTATISTIQUES GÉNÉRALES:")
    print(f"  Nombre total de textes traités: {total_texts}")
    print(f"  Nombre total d'entités prédites: {total_predictions}")
    print(f"  Moyenne d'entités par texte: {total_predictions/total_texts:.2f}")
    
    # Analyse par type d'entité
    df_entities = df_results[df_results['entity'].notna() & (df_results['entity'] != '')]
    
    if not df_entities.empty:
        entity_counts = df_entities['label'].value_counts()
        
        print(f"\nRÉPARTITION PAR TYPE D'ENTITÉ:")
        for entity_type, count in entity_counts.items():
            percentage = (count / total_predictions) * 100
            print(f"  {entity_type}: {count} entités ({percentage:.1f}%)")
        
        # Analyse de la longueur des entités
        entity_lengths = df_entities['entity'].str.len()
        
        print(f"\nANALYSE DE LA LONGUEUR DES ENTITÉS:")
        print(f"  Longueur moyenne: {entity_lengths.mean():.1f} caractères")
        print(f"  Longueur médiane: {entity_lengths.median():.1f} caractères")
        print(f"  Longueur min: {entity_lengths.min()} caractères")
        print(f"  Longueur max: {entity_lengths.max()} caractères")
        
        # Top entités les plus fréquentes par type
        print(f"\nTOP 5 DES ENTITÉS LES PLUS FRÉQUENTES PAR TYPE:")
        for entity_type in ['PER', 'ORG', 'LOC']:
            type_entities = df_entities[df_entities['label'] == entity_type]['entity']
            if not type_entities.empty:
                top_entities = type_entities.value_counts().head(5)
                print(f"\n  {entity_type}:")
                for entity, count in top_entities.items():
                    print(f"    '{entity}': {count} occurrences")
            else:
                print(f"\n  {entity_type}: Aucune entité détectée")
        
        # Analyse de la distribution par texte
        entities_per_text = df_entities.groupby('text_id').size()
        
        print(f"\nDISTRIBUTION DES ENTITÉS PAR TEXTE:")
        print(f"  Textes sans entité: {total_texts - len(entities_per_text)} ({((total_texts - len(entities_per_text))/total_texts)*100:.1f}%)")
        print(f"  Textes avec entités: {len(entities_per_text)} ({(len(entities_per_text)/total_texts)*100:.1f}%)")
        
        if len(entities_per_text) > 0:
            print(f"  Nombre moyen d'entités (textes non-vides): {entities_per_text.mean():.2f}")
            print(f"  Nombre médian d'entités (textes non-vides): {entities_per_text.median():.2f}")
            print(f"  Max entités dans un texte: {entities_per_text.max()}")
        
        # Analyse par type d'entité et par texte
        print(f"\nDISTRIBUTION PAR TYPE ET PAR TEXTE:")
        for entity_type in ['PER', 'ORG', 'LOC']:
            type_data = df_entities[df_entities['label'] == entity_type]
            if not type_data.empty:
                entities_per_text_type = type_data.groupby('text_id').size()
                texts_with_type = len(entities_per_text_type)
                print(f"  {entity_type}: {texts_with_type} textes contiennent ce type ({(texts_with_type/total_texts)*100:.1f}%)")
                print(f"    Moyenne par texte (non-vides): {entities_per_text_type.mean():.2f}")
            else:
                print(f"  {entity_type}: 0 textes contiennent ce type")
    
    else:
        print("ATTENTION: Aucune entité trouvée dans les prédictions!")
    
    return {
        'total_texts': total_texts,
        'total_predictions': total_predictions,
        'entity_counts': entity_counts.to_dict() if not df_entities.empty else {},
        'entities_per_text_stats': entities_per_text.describe().to_dict() if not df_entities.empty else {}
    }

def show_prediction_samples(csv_results_file, n_samples=3):
    """Affiche des échantillons de prédictions pour inspection manuelle"""
    
    print("Chargement du fichier de résultats...")
    
    try:
        df_results = pd.read_csv(csv_results_file, encoding='utf-8')
    except UnicodeDecodeError:
        df_results = pd.read_csv(csv_results_file, encoding='latin1')
    
    # Prendre quelques échantillons de textes avec entités
    df_entities = df_results[df_results['entity'].notna() & (df_results['entity'] != '')]
    
    if df_entities.empty:
        print("Aucune entité trouvée pour afficher des échantillons.")
        return
    
    sample_text_ids = df_entities['text_id'].unique()[:n_samples]
    
    print(f"\nÉCHANTILLONS DE PRÉDICTIONS ({len(sample_text_ids)} textes):")
    print("="*60)
    
    for i, text_id in enumerate(sample_text_ids, 1):
        text_entities = df_entities[df_entities['text_id'] == text_id]
        
        print(f"\nÉchantillon {i} - Text ID: {text_id}")
        print("-" * 30)
        
        # Récupérer le texte original si disponible
        text_row = df_results[df_results['text_id'] == text_id].iloc[0]
        if 'text' in text_row:
            text = str(text_row['text'])
            print(f"Texte ({len(text)} chars): {text[:150]}{'...' if len(text) > 150 else ''}")
        
        print("Entités détectées:")
        for _, entity_row in text_entities.iterrows():
            entity_text = entity_row['entity']
            entity_label = entity_row['label']
            confidence = entity_row.get('confidence', 'N/A')
            print(f"  - {entity_label}: '{entity_text}' (conf: {confidence})")

def analyze_confidence_scores(csv_results_file):
    """Analyse les scores de confiance si disponibles"""
    
    print("Analyse des scores de confiance...")
    
    try:
        df_results = pd.read_csv(csv_results_file, encoding='utf-8')
    except UnicodeDecodeError:
        df_results = pd.read_csv(csv_results_file, encoding='latin1')
    
    if 'confidence' in df_results.columns:
     analyze_confidence_scores('ner_results.csv')   df_entities = df_results[df_results['entity'].notna() & (df_results['entity'] != '')]
        
        if not df_entities.empty and df_entities['confidence'].notna().any():
            confidence_scores = df_entities['confidence'].dropna()
            
            print(f"\nANALYSE DES SCORES DE CONFIANCE:")
            print(f"  Nombre d'entités avec score: {len(confidence_scores)}")
            print(f"  Score moyen: {confidence_scores.mean():.3f}")
            print(f"  Score médian: {confidence_scores.median():.3f}")
            print(f"  Score minimum: {confidence_scores.min():.3f}")
            print(f"  Score maximum: {confidence_scores.max():.3f}")
            print(f"  Écart-type: {confidence_scores.std():.3f}")
            
            # Distribution par type d'entité
            print(f"\nSCORES PAR TYPE D'ENTITÉ:")
            for entity_type in ['PER', 'ORG', 'LOC']:
                type_scores = df_entities[df_entities['label'] == entity_type]['confidence'].dropna()
                if not type_scores.empty:
                    print(f"  {entity_type}: moyenne={type_scores.mean():.3f}, médiane={type_scores.median():.3f}")
                else:
                    print(f"  {entity_type}: Pas de scores disponibles")
            
            # Entités avec faible confiance
            low_confidence = df_entities[df_entities['confidence'] < 0.5]
            if not low_confidence.empty:
                print(f"\nENTITÉS AVEC FAIBLE CONFIANCE (<0.5): {len(low_confidence)}")
                print("Échantillons:")
                for _, row in low_confidence.head(5).iterrows():
                    print(f"  - {row['label']}: '{row['entity']}' (conf: {row['confidence']:.3f})")
        else:
            print("Aucun score de confiance valide trouvé.")
    else:
        print("Colonne 'confidence' non trouvée dans le fichier.")

# Usage
if __name__ == "__main__":
    print("ANALYSE DES PRÉDICTIONS NER - AVANT VÉRITÉ TERRAIN")
    print("="*50)
    
    print("\nPour analyser les prédictions du modèle:")
    print("stats = analyze_predictions_before_ground_truth('ner_results.csv')")
    
    print("\nPour voir des échantillons de prédictions:")
    print("show_prediction_samples('ner_results.csv', n_samples=5)")
    
    print("\nPour analyser les scores de confiance:")
    print("analyze_confidence_scores('ner_results.csv')")

print("Script d'analyse des prédictions NER (avant vérité terrain) prêt")

ANALYSE DES PRÉDICTIONS NER - AVANT VÉRITÉ TERRAIN

Pour analyser les prédictions du modèle:
stats = analyze_predictions_before_ground_truth('ner_results.csv')

Pour voir des échantillons de prédictions:
show_prediction_samples('ner_results.csv', n_samples=5)

Pour analyser les scores de confiance:
analyze_confidence_scores('ner_results.csv')
Script d'analyse des prédictions NER (avant vérité terrain) prêt


In [5]:
stats = analyze_predictions_before_ground_truth('ner_results.csv')

Chargement du fichier de résultats...
Analyse des prédictions du modèle...

STATISTIQUES GÉNÉRALES:
  Nombre total de textes traités: 303
  Nombre total d'entités prédites: 4051
  Moyenne d'entités par texte: 13.37

RÉPARTITION PAR TYPE D'ENTITÉ:
  LOC: 1557 entités (38.4%)
  PER: 1432 entités (35.3%)
  ORG: 1062 entités (26.2%)

ANALYSE DE LA LONGUEUR DES ENTITÉS:
  Longueur moyenne: 14.2 caractères
  Longueur médiane: 13.0 caractères
  Longueur min: 2 caractères
  Longueur max: 78 caractères

TOP 5 DES ENTITÉS LES PLUS FRÉQUENTES PAR TYPE:

  PER:
    'Général de Gaulle': 52 occurrences
    'Maurice PERCHE': 27 occurrences
    'Guy Mollet': 25 occurrences
    'Edmond THORAILLER': 20 occurrences
    'Gérard YVON': 19 occurrences

  ORG:
    'Parti Communiste Français': 81 occurrences
    'Parti Socialiste': 62 occurrences
    'RÉPUBLIQUE FRANÇAISE': 47 occurrences
    'Union de la Gauche': 33 occurrences
    'Parti Communiste': 23 occurrences

  LOC:
    'Eure et Loir': 103 occurrence

In [6]:
show_prediction_samples('ner_results.csv', n_samples=5)

Chargement du fichier de résultats...

ÉCHANTILLONS DE PRÉDICTIONS (5 textes):

Échantillon 1 - Text ID: EL043_L_1967_03_028_03_1_PF_05
------------------------------
Entités détectées:
  - PER: 'Paul Antier' (conf: 0.98235863)
  - PER: 'Antoine de LAYRE' (conf: 0.86638623)
  - PER: 'Antoine de LAYRE' (conf: 0.86842537)
  - LOC: 'Luisant' (conf: 0.70305324)

Échantillon 2 - Text ID: EL009_L_1958_11_028_03_2_PF_02
------------------------------
Entités détectées:
  - PER: 'François A. de MONTFORT' (conf: 0.8047714)
  - LOC: 'Ravensbrück' (conf: 0.9091263)
  - PER: 'Charles de GAULLE' (conf: 0.95995605)
  - LOC: 'Bonneval' (conf: 0.8568747)
  - LOC: 'Illiers' (conf: 0.9163698)
  - LOC: 'Châteaudun' (conf: 0.9182324)
  - LOC: 'Brou' (conf: 0.875824)
  - LOC: 'Cloyes' (conf: 0.9345669)
  - LOC: 'Authon' (conf: 0.9116843)
  - LOC: 'Nogent le Rotrou' (conf: 0.9633028)
  - LOC: 'Thiron' (conf: 0.8277435)
  - LOC: 'La Loupe' (conf: 0.9085297)
  - LOC: 'Dunois' (conf: 0.8644455)
  - LOC: 'Perch

In [7]:
analyze_confidence_scores('ner_results.csv')

Analyse des scores de confiance...

ANALYSE DES SCORES DE CONFIANCE:
  Nombre d'entités avec score: 4051
  Score moyen: 0.856
  Score médian: 0.873
  Score minimum: 0.650
  Score maximum: 1.000
  Écart-type: 0.101

SCORES PAR TYPE D'ENTITÉ:
  PER: moyenne=0.870, médiane=0.892
  ORG: moyenne=0.835, médiane=0.838
  LOC: moyenne=0.856, médiane=0.874


## Métriques après vérification avec vérité terrain

In [8]:
from collections import Counter

def evaluate_metrics_simple(csv_results_file, csv_original_file):
    """Évalue les métriques de performance NER"""
    
    print("Chargement des fichiers...")
    
    # Charger les résultats NER
    try:
        df_results = pd.read_csv(csv_results_file, encoding='utf-8')
    except UnicodeDecodeError:
        df_results = pd.read_csv(csv_results_file, encoding='latin1')
    
    # Charger le CSV original
    try:
        df_original = pd.read_csv(csv_original_file, sep=';', encoding='utf-8')
    except UnicodeDecodeError:
        df_original = pd.read_csv(csv_original_file, sep=';', encoding='latin1')
    
    # Créer la vérité terrain
    print("Création de la vérité terrain...")
    ground_truth = create_ground_truth_simple(df_original)
    
    # Extraire les prédictions
    predictions = extract_predictions_simple(df_results)
    
    # Calculer les métriques
    print("Calcul des métriques...")
    metrics = calculate_metrics_simple(predictions, ground_truth)
    
    # Afficher les résultats
    display_metrics(metrics)
    
    return metrics

def create_ground_truth_simple(df_original):
    """Crée la vérité terrain à partir du CSV original"""
    
    ground_truth = {}
    
    for _, row in df_original.iterrows():
        if pd.notna(row.get('id')) and pd.notna(row.get('texte')):
            # Utiliser l'ID du CSV original comme clé
            text_id = str(row['id'])  # Convertir en string pour cohérence
            text = str(row['texte']).lower()
            
            entities = {'PER': set(), 'ORG': set(), 'LOC': set()}
            
            # Personnes
            if pd.notna(row.get('nom_titulaire')):
                nom = str(row['nom_titulaire']).strip().lower()
                if nom and nom in text:
                    entities['PER'].add(nom)
                    
            if pd.notna(row.get('prenom_titulaire')) and pd.notna(row.get('nom_titulaire')):
                prenom_nom = f"{row['prenom_titulaire']} {row['nom_titulaire']}".strip().lower()
                if prenom_nom and prenom_nom in text:
                    entities['PER'].add(prenom_nom)
            
            if pd.notna(row.get('nom_suppleant')):
                nom = str(row['nom_suppleant']).strip().lower()
                if nom and nom in text:
                    entities['PER'].add(nom)
                    
            if pd.notna(row.get('prenom_suppleant')) and pd.notna(row.get('nom_suppleant')):
                prenom_nom = f"{row['prenom_suppleant']} {row['nom_suppleant']}".strip().lower()
                if prenom_nom and prenom_nom in text:
                    entities['PER'].add(prenom_nom)
            
            # Organisations
            parti_columns = ['parti_titulaire_1', 'parti_titulaire_2', 'parti_titulaire_3', 'parti_titulaire_4', 
                           'parti_suppleant_1', 'parti_suppleant_2', 'parti_suppleant_3', 'parti_suppleant_4', 'parti_suppleant_5']
            
            for col in parti_columns:
                if pd.notna(row.get(col)):
                    parti = str(row[col]).strip().lower()
                    if parti and parti in text:
                        entities['ORG'].add(parti)
            
            # Lieux
            if pd.notna(row.get('departement_nom')):
                dept = str(row['departement_nom']).strip().lower()
                if dept and dept in text:
                    entities['LOC'].add(dept)
            
            lieu_columns = ['lieu_naissance_titulaire', 'lieu_residence_titulaire', 
                           'lieu_naissance_suppleant', 'lieu_residence_suppleant']
            
            for col in lieu_columns:
                if pd.notna(row.get(col)):
                    lieu = str(row[col]).strip().lower()
                    if lieu and lieu in text:
                        entities['LOC'].add(lieu)
            
            ground_truth[text_id] = entities
    
    print(f"Vérité terrain créée pour {len(ground_truth)} textes")
    return ground_truth

def extract_predictions_simple(df_results):
    """Extrait les prédictions du fichier de résultats"""
    
    predictions = {}
    
    # Filtrer les entités non vides
    df_entities = df_results[df_results['entity'].notna() & (df_results['entity'] != '')]
    
    # Récupérer tous les text_id uniques (même ceux sans entités)
    all_text_ids = df_results['text_id'].unique()
    
    for text_id in all_text_ids:
        # Convertir en string pour cohérence avec ground_truth
        text_id_str = str(text_id)
        
        # Filtrer les entités pour ce texte
        text_entities = df_entities[df_entities['text_id'] == text_id]
        
        entities = {'PER': set(), 'ORG': set(), 'LOC': set()}
        
        for _, row in text_entities.iterrows():
            entity_text = str(row['entity']).strip().lower()
            entity_label = row['label']
            
            if entity_label in entities and entity_text:
                entities[entity_label].add(entity_text)
        
        predictions[text_id_str] = entities
    
    print(f"Prédictions extraites pour {len(predictions)} textes")
    return predictions

def calculate_metrics_simple(predictions, ground_truth):
    """Calcule les métriques de performance"""
    
    metrics = {}
    
    # Debug: vérifier la correspondance des IDs
    pred_ids = set(predictions.keys())
    truth_ids = set(ground_truth.keys())
    common_ids = pred_ids & truth_ids
    
    print(f"IDs dans prédictions: {len(pred_ids)}")
    print(f"IDs dans vérité terrain: {len(truth_ids)}")
    print(f"IDs communs: {len(common_ids)}")
    
    if len(common_ids) == 0:
        print("ATTENTION: Aucun ID commun trouvé!")
        print(f"Échantillon prédictions: {list(pred_ids)[:5]}")
        print(f"Échantillon vérité terrain: {list(truth_ids)[:5]}")
    
    for entity_type in ['PER', 'ORG', 'LOC']:
        tp = 0  # True Positives
        fp = 0  # False Positives
        fn = 0  # False Negatives
        
        # Utiliser tous les IDs disponibles
        all_text_ids = pred_ids | truth_ids
        
        for text_id in all_text_ids:
            pred_entities = predictions.get(text_id, {}).get(entity_type, set())
            true_entities = ground_truth.get(text_id, {}).get(entity_type, set())
            
            # True Positives: entités prédites ET dans la vérité terrain
            tp += len(pred_entities & true_entities)
            
            # False Positives: entités prédites mais PAS dans la vérité terrain
            fp += len(pred_entities - true_entities)
            
            # False Negatives: entités dans la vérité terrain mais PAS prédites
            fn += len(true_entities - pred_entities)
        
        # Calcul des métriques
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        metrics[entity_type] = {
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'tp': tp,
            'fp': fp,
            'fn': fn
        }
    
    return metrics

def display_metrics(metrics):
    """Affiche les métriques"""
    
    print("\n" + "="*60)
    print("RÉSULTATS D'ÉVALUATION NER")
    print("="*60)
    
    for entity_type, scores in metrics.items():
        print(f"\n{entity_type} (Entités de type {entity_type}):")
        print(f"  Précision:  {scores['precision']:.3f} ({scores['tp']}/{scores['tp'] + scores['fp']})")
        print(f"  Rappel:     {scores['recall']:.3f} ({scores['tp']}/{scores['tp'] + scores['fn']})")
        print(f"  F1-Score:   {scores['f1_score']:.3f}")
        print(f"  Détails:    TP={scores['tp']}, FP={scores['fp']}, FN={scores['fn']}")
    
    # Score moyen
    avg_precision = sum(s['precision'] for s in metrics.values()) / len(metrics)
    avg_recall = sum(s['recall'] for s in metrics.values()) / len(metrics)
    avg_f1 = sum(s['f1_score'] for s in metrics.values()) / len(metrics)
    
    print(f"\nMOYENNES:")
    print(f"  Précision moyenne: {avg_precision:.3f}")
    print(f"  Rappel moyen:      {avg_recall:.3f}")
    print(f"  F1-Score moyen:    {avg_f1:.3f}")

def debug_matching(csv_results_file, csv_original_file, text_id_sample=None):
    """Fonction de debug pour vérifier la correspondance des entités"""
    
    # Charger les fichiers
    try:
        df_results = pd.read_csv(csv_results_file, encoding='utf-8')
    except UnicodeDecodeError:
        df_results = pd.read_csv(csv_results_file, encoding='latin1')
    
    try:
        df_original = pd.read_csv(csv_original_file, sep=';', encoding='utf-8')
    except UnicodeDecodeError:
        df_original = pd.read_csv(csv_original_file, sep=';', encoding='latin1')
    
    ground_truth = create_ground_truth_simple(df_original)
    predictions = extract_predictions_simple(df_results)
    
    # Si aucun text_id spécifié, prendre un ID commun au hasard
    if text_id_sample is None:
        common_ids = set(predictions.keys()) & set(ground_truth.keys())
        if common_ids:
            import random
            text_id_sample = random.choice(list(common_ids))
        else:
            print("Aucun ID commun trouvé pour le debug!")
            return
    
    text_id_str = str(text_id_sample)
    
    print(f"\nDEBUG pour text_id: {text_id_str}")
    print("="*50)
    
    if text_id_str in ground_truth:
        print("VÉRITÉ TERRAIN:")
        for entity_type, entities in ground_truth[text_id_str].items():
            print(f"  {entity_type}: {entities}")
    else:
        print("VÉRITÉ TERRAIN: Non trouvée")
    
    if text_id_str in predictions:
        print("\nPRÉDICTIONS:")
        for entity_type, entities in predictions[text_id_str].items():
            print(f"  {entity_type}: {entities}")
    else:
        print("\nPRÉDICTIONS: Non trouvées")
    
    # Afficher le texte original si possible
    original_row = df_original[df_original['id'].astype(str) == text_id_str]
    if not original_row.empty:
        text = str(original_row.iloc[0]['texte'])
        print(f"\nTEXTE ORIGINAL ({len(text)} chars):")
        print(text[:200] + "..." if len(text) > 200 else text)

# Usage
if __name__ == "__main__":
    print("ÉVALUATION DES MÉTRIQUES NER")
    print("="*30)
    
    print("\nPour évaluer un fichier de résultats:")
    print("metrics = evaluate_metrics_simple('ner_results.csv', 'Candidatures58-81-290825.csv')")
    
    print("\nPour debugger la correspondance:")
    print("debug_matching('ner_results.csv', 'Candidatures58-81-290825.csv')")

print("Script d'évaluation des métriques prêt")

ÉVALUATION DES MÉTRIQUES NER

Pour évaluer un fichier de résultats:
metrics = evaluate_metrics_simple('ner_results.csv', 'Candidatures58-81-290825.csv')

Pour debugger la correspondance:
debug_matching('ner_results.csv', 'Candidatures58-81-290825.csv')
Script d'évaluation des métriques prêt


In [9]:
metrics = evaluate_metrics_simple('ner_results.csv', 'Candidatures58-81-290825.csv')

Chargement des fichiers...
Création de la vérité terrain...
Vérité terrain créée pour 303 textes
Prédictions extraites pour 303 textes
Calcul des métriques...
IDs dans prédictions: 303
IDs dans vérité terrain: 303
IDs communs: 303

RÉSULTATS D'ÉVALUATION NER

PER (Entités de type PER):
  Précision:  0.429 (456/1063)
  Rappel:     0.410 (456/1112)
  F1-Score:   0.419
  Détails:    TP=456, FP=607, FN=656

ORG (Entités de type ORG):
  Précision:  0.154 (137/887)
  Rappel:     0.537 (137/255)
  F1-Score:   0.240
  Détails:    TP=137, FP=750, FN=118

LOC (Entités de type LOC):
  Précision:  0.151 (174/1155)
  Rappel:     0.274 (174/634)
  F1-Score:   0.195
  Détails:    TP=174, FP=981, FN=460

MOYENNES:
  Précision moyenne: 0.245
  Rappel moyen:      0.407
  F1-Score moyen:    0.285


In [10]:
debug_matching('ner_results.csv', 'Candidatures58-81-290825.csv')

Vérité terrain créée pour 303 textes
Prédictions extraites pour 303 textes

DEBUG pour text_id: EL103_L_1978_03_041_03_1_PF_03
VÉRITÉ TERRAIN:
  PER: {'girard', 'bernard hemme', 'hemme', 'pierre girard'}
  ORG: set()
  LOC: {'loir-et-cher', 'saint-arnoult', 'vendôme'}

PRÉDICTIONS:
  PER: {'bernard hemme', 'pierre girard'}
  ORG: {'république française département de loir et cher', 'parti communiste'}
  LOC: {'la ville aux clercs', 'saint arnoult', 'vendômois', 'vendôme', 'france'}

TEXTE ORIGINAL (5818 chars):



RÉPUBLIQUE FRANÇAISE - DÉPARTEMENT DE LOIR-ET-CHER

CIRCONSCRIPTION DE VENDÔME



ÉLECTIONS LÉGISLATIVES DU 12 MARS 1978



MADAME, MADEMOISELLE, MONSIEUR,

Le 12 mars et le 19 éventuellement, vous...
